In [29]:
!pip install cryptography scikit-learn fpdf tabulate ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=cfd84b0a683e13a1b78ac740bbf3606078c9429755783ed041614cb287902e19
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [30]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [31]:
import sys
sys.path.append('/content/FortiCryptX-Colab')


In [32]:
from key_management.key_manager import generate_rsa_key_pair, save_keys_to_files, load_private_key, load_public_key

# Generate key pair
private_key, public_key = generate_rsa_key_pair()

# Save keys to disk
save_keys_to_files(private_key, public_key, "private.pem", "public.pem")

# Load them back
private_key = load_private_key("private.pem")
public_key = load_public_key("public.pem")


In [33]:
from crypto_engine.hybrid_crypto import encrypt_file, decrypt_file
from audit_logger.logger import log_event
import time
import os

# Write a sample plaintext file
with open("plain.txt", "w") as f:
    f.write("This is a secure test file.")

# --- ENCRYPTION ---
start = time.time()
encrypt_file("plain.txt", public_key, "encrypted.dat")
enc_duration = time.time() - start

log_event("encryption", {
    "file": "plain.txt",
    "encrypted_as": "encrypted.dat",
    "duration": enc_duration,
    "key_size": 2048,
    "file_size": os.path.getsize("plain.txt")
})

# --- DECRYPTION ---
start = time.time()
decrypt_file("encrypted.dat", private_key, "decrypted.txt")
dec_duration = time.time() - start

log_event("decryption", {
    "file": "encrypted.dat",
    "decrypted_as": "decrypted.txt",
    "duration": dec_duration,
    "key_size": 2048,
    "file_size": os.path.getsize("encrypted.dat")
})

# Verify decrypted output
with open("decrypted.txt") as f:
    print("Decrypted content:", f.read())


Decrypted content: This is a secure test file.


In [34]:
from security_tools.hash_integrity import compute_sha256

def verify_integrity(original, decrypted):
    hash1 = compute_sha256(original)
    hash2 = compute_sha256(decrypted)
    print(f"Integrity Match: {hash1 == hash2}")


In [35]:
import importlib
import audit_logger.logger
importlib.reload(audit_logger.logger)

from audit_logger.logger import log_event, read_logs


In [36]:
from audit_logger.logger import log_event, read_logs

log_event("encryption", {
    "file": "plain.txt",
    "encrypted_as": "encrypted.dat",
    "duration": 1.23,
    "key_size": 2048,
    "file_size": 12345
})

print(read_logs())



[{'timestamp': '2025-05-25T11:15:04.817930', 'event_type': 'encryption', 'details': {'file': 'plain.txt', 'encrypted_as': 'encrypted.dat', 'duration': 0.00839686393737793, 'key_size': 2048, 'file_size': 27}}, {'timestamp': '2025-05-25T11:15:04.821593', 'event_type': 'decryption', 'details': {'file': 'encrypted.dat', 'decrypted_as': 'decrypted.txt', 'duration': 0.0031273365020751953, 'key_size': 2048, 'file_size': 308}}, {'timestamp': '2025-05-25T11:15:14.599390', 'event_type': 'encryption', 'details': {'file': 'plain.txt', 'encrypted_as': 'encrypted.dat', 'duration': 1.23, 'key_size': 2048, 'file_size': 12345}}]


In [37]:
from benchmarking.benchmark import benchmark_encryption_algorithms

results = benchmark_encryption_algorithms("plain.txt", public_key, private_key)
print("Benchmark results:", results)


Benchmark results: {'encryption_time': 0.0005490779876708984, 'decryption_time': 0.0017087459564208984, 'file_size_kb': 0.0263671875}


In [38]:
from key_management.key_audit import train_anomaly_model, detect_anomaly
from audit_logger.logger import LOG_FILE

train_anomaly_model(LOG_FILE)
anomalies = detect_anomaly(LOG_FILE)
print("Anomalies found:")
print(anomalies)


Anomalies found:
        file   encrypted_as  duration  key_size  file_size decrypted_as  \
2  plain.txt  encrypted.dat      1.23      2048      12345          NaN   

   anomaly  
2       -1  


In [39]:
from audit_logger.logger import read_logs
from IPython.display import Markdown, display

def view_logs():
    logs = read_logs()
    if not logs:
        print("📭 No logs found.")
        return

    for entry in logs:
        display(Markdown(f"""
### 🕒 {entry['timestamp']}
**Type:** {entry['event_type']}
**Details:** `{entry['details']}`
---
"""))


In [40]:
import ipywidgets as widgets
from IPython.display import display
from crypto_engine.hybrid_crypto import encrypt_file, decrypt_file
from key_management.key_manager import load_public_key, load_private_key
from security_tools.hash_integrity import compute_sha256

# Load keys
public_key = load_public_key("public.pem")
private_key = load_private_key("private.pem")

def encrypt_wrapper():
    file_input = widgets.Text(value='plain.txt', description='Input File:')
    file_output = widgets.Text(value='encrypted.dat', description='Output File:')
    button = widgets.Button(description="🔐 Encrypt")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            try:
                encrypt_file(file_input.value, public_key, file_output.value)
                print(f"✅ Encrypted: {file_input.value} → {file_output.value}")
            except Exception as e:
                print(f"❌ Encryption error: {e}")

    button.on_click(on_click)
    display(widgets.VBox([file_input, file_output, button, output]))

def decrypt_wrapper():
    file_input = widgets.Text(value='encrypted.dat', description='Input File:')
    file_output = widgets.Text(value='decrypted.txt', description='Output File:')
    original_file = widgets.Text(value='plain.txt', description='Compare With:')
    button = widgets.Button(description="🔓 Decrypt + Verify")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            try:
                decrypt_file(file_input.value, private_key, file_output.value)
                print(f"✅ Decrypted: {file_input.value} → {file_output.value}")

                # Integrity check
                hash1 = compute_sha256(original_file.value)
                hash2 = compute_sha256(file_output.value)
                print("🔐 Integrity Check:", "✅ Match" if hash1 == hash2 else "❌ Mismatch")
            except Exception as e:
                print(f"❌ Decryption error: {e}")

    button.on_click(on_click)
    display(widgets.VBox([file_input, file_output, original_file, button, output]))


In [41]:
from cloud_integration.gdrive_utils import upload_to_drive, download_from_drive

def upload_widget():
    file_input = widgets.Text(value='encrypted.dat', description='File to Upload:')
    button = widgets.Button(description="⬆️ Upload to Drive")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            try:
                upload_to_drive(file_input.value)
            except Exception as e:
                print(f"❌ Upload error: {e}")

    button.on_click(on_click)
    display(widgets.VBox([file_input, button, output]))


In [42]:
from ui.colab_ui import show_main_ui

def launch_workflow_ui():
    show_main_ui(
        encrypt_callback=encrypt_wrapper,
        decrypt_callback=decrypt_wrapper,
        view_logs_callback=view_logs
    )


In [44]:
launch_workflow_ui()


Output()

# Creating the project folder structure and root **files**

In [1]:
import os

# Root folder
base_dir = "/content/FortiCryptX-Colab"

# Define subdirectories
folders = [
    "crypto_engine",
    "key_management",
    "security_tools",
    "cloud_integration",
    "ui",
    "audit_logger",
    "pq_simulation",
    "benchmarking"
]

# Create folders
for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

root_files = ["main.ipynb", "requirements.txt", "README.md"]
for file in root_files:
    open(os.path.join(base_dir, file), 'a').close()

print("Directory structure created.")


Directory structure created.


# Creating __init__.py in each **subfolder**

In [2]:
# Create __init__.py in each subfolder
for folder in folders:
    init_path = os.path.join(base_dir, folder, "__init__.py")
    with open(init_path, "w") as f:
        f.write("# Init for " + folder)


# **crypto_engine**

hybrid_crypto.py

In [3]:
%%writefile /content/FortiCryptX-Colab/crypto_engine/hybrid_crypto.py

import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sympadding
from cryptography.hazmat.primitives.asymmetric import padding as asympadding
from cryptography.hazmat.primitives import hashes

def encrypt_file(file_path, public_key, output_path):
    aes_key = os.urandom(32)
    iv = os.urandom(16)

    with open(file_path, "rb") as f:
        data = f.read()

    padder = sympadding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    encrypted_key = public_key.encrypt(
        aes_key,
        asympadding.OAEP(
            mgf=asympadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(output_path, "wb") as f:
        f.write(len(encrypted_key).to_bytes(4, 'big'))
        f.write(encrypted_key)
        f.write(iv)
        f.write(encrypted_data)

def decrypt_file(enc_file_path, private_key, output_path):
    with open(enc_file_path, "rb") as f:
        key_len = int.from_bytes(f.read(4), 'big')
        encrypted_key = f.read(key_len)
        iv = f.read(16)
        encrypted_data = f.read()

    aes_key = private_key.decrypt(
        encrypted_key,
        asympadding.OAEP(
            mgf=asympadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    unpadder = sympadding.PKCS7(128).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()

    with open(output_path, "wb") as f:
        f.write(data)


Writing /content/FortiCryptX-Colab/crypto_engine/hybrid_crypto.py


ecies_crypto.py

In [4]:
%%writefile /content/FortiCryptX-Colab/crypto_engine/ecies_crypto.py

import os
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def ecies_encrypt(public_key, data):
    ephemeral_key = ec.generate_private_key(ec.SECP384R1())
    shared_secret = ephemeral_key.exchange(ec.ECDH(), public_key)

    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies"
    ).derive(shared_secret)

    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padding_len = 16 - len(data) % 16
    data += bytes([padding_len]) * padding_len
    ciphertext = encryptor.update(data) + encryptor.finalize()

    return ephemeral_key.public_key(), iv, ciphertext

def ecies_decrypt(private_key, peer_public_key, iv, ciphertext):
    shared_secret = private_key.exchange(ec.ECDH(), peer_public_key)

    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies"
    ).derive(shared_secret)

    cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()

    padding_len = padded_data[-1]
    return padded_data[:-padding_len]


Writing /content/FortiCryptX-Colab/crypto_engine/ecies_crypto.py


pbe_crypto.py

In [5]:
%%writefile /content/FortiCryptX-Colab/crypto_engine/pbe_crypto.py

import os
import base64
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes

def derive_key_pbkdf2(password: str, salt: bytes = None, iterations: int = 100_000):
    if salt is None:
        salt = os.urandom(16)

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
    )

    key = kdf.derive(password.encode())
    return key, salt

def verify_key_pbkdf2(password: str, salt: bytes, key: bytes, iterations: int = 100_000):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
    )
    try:
        kdf.verify(password.encode(), key)
        return True
    except Exception:
        return False


Writing /content/FortiCryptX-Colab/crypto_engine/pbe_crypto.py


digital_signatures.py

In [6]:
%%writefile /content/FortiCryptX-Colab/crypto_engine/digital_signatures.py

from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding

def sign_file(file_path, private_key):
    with open(file_path, "rb") as f:
        data = f.read()

    signature = private_key.sign(
        data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def verify_signature(file_path, signature, public_key):
    with open(file_path, "rb") as f:
        data = f.read()

    try:
        public_key.verify(
            signature,
            data,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False


Writing /content/FortiCryptX-Colab/crypto_engine/digital_signatures.py


entropy_analysis.py


In [7]:
%%writefile /content/FortiCryptX-Colab/crypto_engine/entropy_analysis.py

import math

def calculate_entropy(data: bytes) -> float:
    if not data:
        return 0.0

    freq = {}
    for b in data:
        freq[b] = freq.get(b, 0) + 1

    entropy = 0
    length = len(data)
    for count in freq.values():
        p = count / length
        entropy -= p * math.log2(p)

    return entropy

def analyze_file_entropy(file_path: str, threshold: float = 7.5) -> dict:
    with open(file_path, "rb") as f:
        data = f.read()

    entropy = calculate_entropy(data)
    result = {
        "entropy": entropy,
        "is_suspicious": entropy >= threshold
    }
    return result


Writing /content/FortiCryptX-Colab/crypto_engine/entropy_analysis.py


# **key_management**

key_manager.py

In [8]:
%%writefile /content/FortiCryptX-Colab/key_management/key_manager.py

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

def generate_rsa_key_pair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

def save_keys_to_files(private_key, public_key, priv_path, pub_path):
    with open(priv_path, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open(pub_path, "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

def load_private_key(path):
    with open(path, "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None)

def load_public_key(path):
    with open(path, "rb") as f:
        return serialization.load_pem_public_key(f.read())


Writing /content/FortiCryptX-Colab/key_management/key_manager.py


key_audit.py

In [9]:
%%writefile /content/FortiCryptX-Colab/key_management/key_audit.py
import pandas as pd
from sklearn.ensemble import IsolationForest
import joblib
import json

MODEL_PATH = "/content/FortiCryptX-Colab/key_management/anomaly_model.pkl"

def train_anomaly_model(log_file):
    rows = []
    with open(log_file, 'r') as f:
        for line in f:
            try:
                obj = json.loads(line.strip())
                if 'details' in obj:
                    rows.append(obj['details'])
            except json.JSONDecodeError:
                print(f"[!] Skipping bad line: {line.strip()}")

    df = pd.DataFrame(rows)

    # Ensure required columns exist
    if not all(col in df.columns for col in ['file_size', 'duration', 'key_size']):
        raise ValueError("Missing required fields (file_size, duration, key_size) in logs.")

    model = IsolationForest(contamination=0.05)
    model.fit(df[['file_size', 'duration', 'key_size']])
    joblib.dump(model, MODEL_PATH)


def detect_anomaly(log_file):
    model = joblib.load(MODEL_PATH)
    rows = []
    with open(log_file, 'r') as f:
        for line in f:
            try:
                obj = json.loads(line.strip())
                if 'details' in obj:
                    rows.append(obj['details'])
            except json.JSONDecodeError:
                continue

    df = pd.DataFrame(rows)
    preds = model.predict(df[['file_size', 'duration', 'key_size']])
    df['anomaly'] = preds
    return df[df['anomaly'] == -1]


Writing /content/FortiCryptX-Colab/key_management/key_audit.py


# **security_tools**

hash_integrity.py

In [10]:
%%writefile /content/FortiCryptX-Colab/security_tools/hash_integrity.py
import hashlib
import os

def compute_sha256(filepath):
    sha256 = hashlib.sha256()
    with open(filepath, "rb") as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

def verify_file_integrity(original_file, decrypted_file):
    return compute_sha256(original_file) == compute_sha256(decrypted_file)


Writing /content/FortiCryptX-Colab/security_tools/hash_integrity.py


file_shredder.py

In [11]:
%%writefile /content/FortiCryptX-Colab/security_tools/file_shredder.py
import os
import random

def shred_file(filepath, passes=3):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"{filepath} not found.")

    filesize = os.path.getsize(filepath)

    with open(filepath, "ba+", buffering=0) as f:
        for i in range(passes):
            f.seek(0)
            f.write(os.urandom(filesize))
            f.flush()
            os.fsync(f.fileno())

    os.remove(filepath)
    print(f"{filepath} shredded securely with {passes} overwrite passes.")


Writing /content/FortiCryptX-Colab/security_tools/file_shredder.py


tamper_detection.py

In [12]:
%%writefile /content/FortiCryptX-Colab/security_tools/tamper_detection.py
import json
import os
from .hash_integrity import compute_sha256

def save_file_hash(filepath, hash_db_path="hashes.json"):
    file_hash = compute_sha256(filepath)
    hashes = {}

    if os.path.exists(hash_db_path):
        with open(hash_db_path, "r") as f:
            hashes = json.load(f)

    hashes[os.path.basename(filepath)] = file_hash

    with open(hash_db_path, "w") as f:
        json.dump(hashes, f, indent=2)

def verify_file_before_decryption(filepath, hash_db_path="hashes.json"):
    if not os.path.exists(hash_db_path):
        raise FileNotFoundError("Hash database not found.")

    with open(hash_db_path, "r") as f:
        hashes = json.load(f)

    filename = os.path.basename(filepath)
    if filename not in hashes:
        raise ValueError(f"No hash record found for {filename}.")

    current_hash = compute_sha256(filepath)
    expected_hash = hashes[filename]

    return current_hash == expected_hash


Writing /content/FortiCryptX-Colab/security_tools/tamper_detection.py


# **cloud_integration**

gdrive_utils.py

In [13]:
%%writefile /content/FortiCryptX-Colab/cloud_integration/gdrive_utils.py
import shutil
import os

def upload_to_drive(local_path, drive_path="/content/drive/MyDrive/FortiCryptX-Uploads"):
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"{local_path} not found.")

    os.makedirs(drive_path, exist_ok=True)
    filename = os.path.basename(local_path)
    destination = os.path.join(drive_path, filename)
    shutil.copy2(local_path, destination)
    print(f"Uploaded: {filename} to Google Drive -> {drive_path}")

def download_from_drive(drive_filename, drive_path="/content/drive/MyDrive/FortiCryptX-Uploads", download_dir="/content"):
    source_path = os.path.join(drive_path, drive_filename)
    if not os.path.exists(source_path):
        raise FileNotFoundError(f"{drive_filename} not found in Drive folder.")

    destination = os.path.join(download_dir, drive_filename)
    shutil.copy2(source_path, destination)
    print(f"Downloaded: {drive_filename} to {download_dir}")
    return destination


Writing /content/FortiCryptX-Colab/cloud_integration/gdrive_utils.py


file_picker.py

In [14]:
%%writefile /content/FortiCryptX-Colab/cloud_integration/file_picker.py
from google.colab import drive
import ipywidgets as widgets
import os

def mount_drive():
    drive.mount('/content/drive')

def select_file_from_drive():
    files = [f for f in os.listdir("/content/drive/MyDrive") if not f.startswith('.')]
    dropdown = widgets.Dropdown(options=files, description='Select file:')
    display(dropdown)
    return dropdown


Writing /content/FortiCryptX-Colab/cloud_integration/file_picker.py


session_persistence.py

In [15]:
%%writefile /content/FortiCryptX-Colab/cloud_integration/session_persistence.py
import os
import shutil

PERSIST_DIR = "/content/drive/MyDrive/FortiCryptX-Persist"

def save_file_session(local_path):
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"{local_path} does not exist.")

    os.makedirs(PERSIST_DIR, exist_ok=True)
    filename = os.path.basename(local_path)
    destination = os.path.join(PERSIST_DIR, filename)
    shutil.copy2(local_path, destination)
    print(f"Persisted {filename} to Drive.")

def load_file_session(filename):
    filepath = os.path.join(PERSIST_DIR, filename)
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"{filename} not found in persistent storage.")

    local_copy = os.path.join("/content", filename)
    shutil.copy2(filepath, local_copy)
    print(f"Loaded {filename} from persistent storage.")
    return local_copy

def list_persisted_files():
    if not os.path.exists(PERSIST_DIR):
        return []

    return os.listdir(PERSIST_DIR)


Writing /content/FortiCryptX-Colab/cloud_integration/session_persistence.py


# **ui**

colab_ui.py

In [16]:
%%writefile /content/FortiCryptX-Colab/ui/colab_ui.py
import ipywidgets as widgets
from IPython.display import display

def show_main_ui(encrypt_callback, decrypt_callback, view_logs_callback):
    encrypt_btn = widgets.Button(description="🔐 Encrypt File")
    decrypt_btn = widgets.Button(description="🔓 Decrypt File")
    view_logs_btn = widgets.Button(description="📜 View Logs")

    output = widgets.Output()

    def on_encrypt_clicked(b):
        with output:
            output.clear_output()
            encrypt_callback()

    def on_decrypt_clicked(b):
        with output:
            output.clear_output()
            decrypt_callback()

    def on_logs_clicked(b):
        with output:
            output.clear_output()
            view_logs_callback()

    encrypt_btn.on_click(on_encrypt_clicked)
    decrypt_btn.on_click(on_decrypt_clicked)
    view_logs_btn.on_click(on_logs_clicked)

    display(widgets.HBox([encrypt_btn, decrypt_btn, view_logs_btn]))
    display(output)


Writing /content/FortiCryptX-Colab/ui/colab_ui.py


user_simulation.py

In [17]:
%%writefile /content/FortiCryptX-Colab/ui/user_simulation.py
import ipywidgets as widgets
from IPython.display import display

USER_PROFILES = {
    "User A": {"email": "usera@example.com", "role": "admin"},
    "User B": {"email": "userb@example.com", "role": "viewer"}
}

def simulate_user():
    dropdown = widgets.Dropdown(
        options=list(USER_PROFILES.keys()),
        description="User Profile:"
    )

    output = widgets.Output()

    def on_change(change):
        with output:
            output.clear_output()
            selected = change["new"]
            profile = USER_PROFILES[selected]
            print(f"Selected: {selected}")
            print(f"Email: {profile['email']}")
            print(f"Role: {profile['role']}")

    dropdown.observe(on_change, names="value")
    display(dropdown, output)
    return dropdown


Writing /content/FortiCryptX-Colab/ui/user_simulation.py


challenge_response.py

In [18]:
%%writefile /content/FortiCryptX-Colab/ui/challenge_response.py
import ipywidgets as widgets
from IPython.display import display

def challenge_response_auth(expected_answer, callback_on_success):
    question = "What is your decryption passphrase?"
    input_box = widgets.Text(description="Passphrase:", placeholder='Enter passphrase...', layout=widgets.Layout(width='50%'))
    button = widgets.Button(description="✔️ Submit")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            if input_box.value == expected_answer:
                print("✅ Authenticated. Proceeding...")
                callback_on_success()
            else:
                print("❌ Incorrect passphrase.")

    button.on_click(on_click)
    display(widgets.VBox([widgets.Label(question), input_box, button, output]))


Writing /content/FortiCryptX-Colab/ui/challenge_response.py


# **audit_logger**

logger.py

In [19]:
%%writefile /content/FortiCryptX-Colab/audit_logger/logger.py
import json
from datetime import datetime
import os

LOG_FILE = "/content/FortiCryptX-Colab/audit_logger/logs.json"

def log_event(event_type, details):
    entry = {
        "timestamp": datetime.utcnow().isoformat(),
        "event_type": event_type,
        "details": details
    }
    with open(LOG_FILE, "a") as f:
        json.dump(entry, f)
        f.write("\n")

def read_logs():
    if not os.path.exists(LOG_FILE):
        return []
    logs = []
    with open(LOG_FILE, "r") as f:
        for line in f:
            try:
                logs.append(json.loads(line.strip()))
            except json.JSONDecodeError:
                continue
    return logs


Writing /content/FortiCryptX-Colab/audit_logger/logger.py


blockchain_logger.py

In [20]:
%%writefile /content/FortiCryptX-Colab/audit_logger/blockchain_logger.py
import hashlib
import time
import json
import os

BLOCKCHAIN_FILE = "/content/FortiCryptX-Colab/audit_logger/blockchain.json"

def compute_block_hash(block):
    block_string = json.dumps(block, sort_keys=True).encode()
    return hashlib.sha256(block_string).hexdigest()

def get_last_block():
    if not os.path.exists(BLOCKCHAIN_FILE) or os.stat(BLOCKCHAIN_FILE).st_size == 0:
        return None
    with open(BLOCKCHAIN_FILE, "r") as f:
        chain = json.load(f)
        return chain[-1]

def create_block(event_type, data):
    previous_block = get_last_block()
    previous_hash = compute_block_hash(previous_block) if previous_block else "0" * 64
    block = {
        "timestamp": time.time(),
        "event_type": event_type,
        "data": data,
        "previous_hash": previous_hash
    }
    block["hash"] = compute_block_hash(block)
    return block

def add_block_to_chain(event_type, data):
    block = create_block(event_type, data)
    if not os.path.exists(BLOCKCHAIN_FILE):
        chain = []
    else:
        with open(BLOCKCHAIN_FILE, "r") as f:
            try:
                chain = json.load(f)
            except json.JSONDecodeError:
                chain = []
    chain.append(block)
    with open(BLOCKCHAIN_FILE, "w") as f:
        json.dump(chain, f, indent=2)


Writing /content/FortiCryptX-Colab/audit_logger/blockchain_logger.py


report_generator.py

In [21]:
%%writefile /content/FortiCryptX-Colab/audit_logger/report_generator.py
import markdown
import json
from datetime import datetime
from fpdf import FPDF

LOG_FILE = "/content/FortiCryptX-Colab/audit_logger/logs.json"

def generate_markdown_report(output_path="/content/audit_report.md"):
    with open(LOG_FILE) as f:
        logs = [json.loads(line) for line in f.readlines()]

    md_lines = ["# 🔍 FortiCryptX Audit Report", f"**Generated:** {datetime.utcnow().isoformat()}", "---"]
    for log in logs:
        md_lines.append(f"### 🕒 {log['timestamp']}")
        md_lines.append(f"**Type:** {log['event_type']}")
        md_lines.append(f"**Details:** `{log['details']}`")
        md_lines.append("---")

    with open(output_path, "w") as f:
        f.write("\n".join(md_lines))

def generate_pdf_report(md_path="/content/audit_report.md", pdf_path="/content/audit_report.pdf"):
    with open(md_path, "r") as f:
        lines = f.readlines()

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    for line in lines:
        clean_line = line.strip().replace("**", "").replace("#", "").replace("`", "")
        if clean_line:
            pdf.multi_cell(0, 10, clean_line)

    pdf.output(pdf_path)


Writing /content/FortiCryptX-Colab/audit_logger/report_generator.py


# **pq_simulation**

pq_hybrid.py

In [22]:
%%writefile /content/FortiCryptX-Colab/pq_simulation/pq_hybrid.py
import os
import base64
from cryptography.hazmat.primitives.asymmetric import rsa, padding as asympadding
from cryptography.hazmat.primitives import hashes

# Simulated PQ encryption (mocked with XOR and base64)
def pq_encrypt(message, pq_key):
    return base64.b64encode(bytes([b ^ pq_key[i % len(pq_key)] for i, b in enumerate(message)]))

def pq_decrypt(ciphertext, pq_key):
    data = base64.b64decode(ciphertext)
    return bytes([b ^ pq_key[i % len(pq_key)] for i, b in enumerate(data)])

def hybrid_encrypt(message, rsa_public_key, pq_key):
    rsa_encrypted = rsa_public_key.encrypt(
        message,
        asympadding.OAEP(
            mgf=asympadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    pq_encrypted = pq_encrypt(rsa_encrypted, pq_key)
    return pq_encrypted

def hybrid_decrypt(ciphertext, rsa_private_key, pq_key):
    rsa_encrypted = pq_decrypt(ciphertext, pq_key)
    decrypted = rsa_private_key.decrypt(
        rsa_encrypted,
        asympadding.OAEP(
            mgf=asympadding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted


Writing /content/FortiCryptX-Colab/pq_simulation/pq_hybrid.py


pq_key_exchange.py

In [23]:
%%writefile /content/FortiCryptX-Colab/pq_simulation/pq_key_exchange.py
import os
import hashlib

# Simulate Kyber-style key encapsulation (using shared random keys)
def generate_pq_keys():
    private_key = os.urandom(32)
    public_key = hashlib.sha256(private_key).digest()
    return public_key, private_key

def encapsulate_key(public_key):
    shared_secret = os.urandom(32)
    encrypted_secret = bytes([b ^ public_key[i % len(public_key)] for i, b in enumerate(shared_secret)])
    return shared_secret, encrypted_secret

def decapsulate_key(encrypted_secret, private_key):
    public_key = hashlib.sha256(private_key).digest()
    shared_secret = bytes([b ^ public_key[i % len(public_key)] for i, b in enumerate(encrypted_secret)])
    return shared_secret


Writing /content/FortiCryptX-Colab/pq_simulation/pq_key_exchange.py


threat_model.py

In [24]:
%%writefile /content/FortiCryptX-Colab/pq_simulation/threat_model.py
import time
import random

def benchmark_symmetric():
    start = time.time()
    _ = [random.getrandbits(128) for _ in range(100000)]
    return time.time() - start

def benchmark_rsa():
    time.sleep(0.3)  # Simulated delay
    return 0.3

def benchmark_pq():
    time.sleep(0.5)  # Simulated PQ algorithm latency
    return 0.5

def run_threat_comparison():
    symmetric = benchmark_symmetric()
    rsa = benchmark_rsa()
    pq = benchmark_pq()

    print("🔐 Threat Model Benchmark:")
    print(f"Symmetric (AES-like) Ops Time: {symmetric:.4f}s")
    print(f"Traditional RSA Ops Time    : {rsa:.4f}s")
    print(f"Post-Quantum (PQ Sim) Time  : {pq:.4f}s")

    print("\n📉 Relative Resistance Summary:")
    print(" - RSA: Vulnerable to Shor’s Algorithm (Quantum threat)")
    print(" - PQ (Kyber/Dilithium): Resistant to known quantum attacks")
    print(" - AES: Key-length based resistance; 256-bit recommended")

if __name__ == "__main__":
    run_threat_comparison()


Writing /content/FortiCryptX-Colab/pq_simulation/threat_model.py


# **benchmarking**

benchmark.py

In [25]:
%%writefile /content/FortiCryptX-Colab/benchmarking/benchmark.py
import time
import os
from crypto_engine.hybrid_crypto import encrypt_file, decrypt_file

def benchmark_encryption_algorithms(input_file, public_key, private_key):
    results = {}
    start = time.time()
    encrypt_file(input_file, public_key, "test.enc")
    end = time.time()
    results["encryption_time"] = end - start

    start = time.time()
    decrypt_file("test.enc", private_key, "test.dec")
    end = time.time()
    results["decryption_time"] = end - start

    results["file_size_kb"] = os.path.getsize(input_file) / 1024
    return results

def generate_benchmark_report():
    print("Benchmarking complete. Compare times and file sizes in your notebook output.")


Writing /content/FortiCryptX-Colab/benchmarking/benchmark.py


threat_table.py

In [26]:
%%writefile /content/FortiCryptX-Colab/benchmarking/threat_table.py
from tabulate import tabulate

def get_threat_summary():
    threats = [
        {
            "Threat Vector": "Brute Force",
            "Target": "AES Key / Password",
            "Mitigation": "256-bit AES + PBKDF2/bcrypt with salt and iterations"
        },
        {
            "Threat Vector": "Key Interception",
            "Target": "AES Key in Transit",
            "Mitigation": "ECIES Hybrid Encryption, ECDH Key Exchange"
        },
        {
            "Threat Vector": "Quantum Attacks",
            "Target": "RSA Encryption",
            "Mitigation": "Simulated PQ Hybrid Encryption (Kyber/Dilithium)"
        },
        {
            "Threat Vector": "File Tampering",
            "Target": "Stored Files",
            "Mitigation": "SHA-256 Integrity Verification + Tamper Detection"
        },
        {
            "Threat Vector": "Unauthorized Access",
            "Target": "Files/Keys",
            "Mitigation": "Challenge-Response Auth + Session Persistence"
        },
        {
            "Threat Vector": "Forensic Recovery",
            "Target": "Deleted Sensitive Files",
            "Mitigation": "Multi-pass File Shredder (DoD standard)"
        },
        {
            "Threat Vector": "Audit Manipulation",
            "Target": "Logs/History",
            "Mitigation": "Immutable JSON Logs + Optional Blockchain Anchoring"
        },
        {
            "Threat Vector": "Cloud Leakage",
            "Target": "GDrive Files",
            "Mitigation": "On-device Encryption + Manual Upload/Download"
        }
    ]
    return tabulate(threats, headers="keys", tablefmt="grid")

if __name__ == "__main__":
    print(get_threat_summary())


Writing /content/FortiCryptX-Colab/benchmarking/threat_table.py


In [27]:
!pip install cryptography

import os
import sys

# Create directory structure if not present
base_dir = "/content/FortiCryptX-Colab"
sys.path.append(base_dir)

folders = [
    "crypto_engine", "key_management", "security_tools",
    "cloud_integration", "ui", "audit_logger",
    "pq_simulation", "benchmarking"
]

for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)
    open(os.path.join(base_dir, folder, "__init__.py"), 'w').close()


In [28]:
%%writefile /content/FortiCryptX-Colab/main.ipynb
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# FortiCryptX-Colab: Main Entry Point\\n",
    "from crypto_engine.hybrid_crypto import encrypt_file, decrypt_file\\n",
    "from key_management.key_manager import generate_keys\\n",
    "from security_tools.hash_integrity import compute_sha256, verify_file_integrity\\n",
    "from audit_logger.logger import log_event\\n",
    "from ui.colab_ui import launch_ui\\n",
    "\\n",
    "# Initial setup\\n",
    "print(\\\"🔐 FortiCryptX-Colab Loaded. Use launch_ui() to begin.\\\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
"""

with open("/content/write_main_script.py", "w") as f:
    f.write("%%writefile /content/FortiCryptX-Colab/main.ipynb\n")
    f.write(notebook_code.split('\n', 1)[1])  # Skip first line already included


Overwriting /content/FortiCryptX-Colab/main.ipynb
